In [1]:
import numpy as np
from skimage.util.shape import view_as_windows
from numpy.lib.stride_tricks import as_strided

##### Input Layer

In [2]:
input_data_list = [[-1, 1, 1, 1,-1],
                   [-1, 1,-1, 1,-1],
                   [-1, 1, 1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1, 1,-1,-1],
                   [-1, 1,-1,-1,-1]]

# Convert list to input data matrix
input_data_matrix = np.asarray(a=input_data_list, dtype=np.float32)

print(input_data_matrix.shape)

(7, 5)


##### Convolution Layer(convolution operation)

In [3]:
box_filter_list = [[1, 1, 1],
                   [1,-1, 1],
                   [1, 1, 1]]

vertical_line_filter_list = [[-1, 1,-1],
                      [-1, 1,-1],
                      [-1, 1,-1]]

diagonal_line_filter_list = [[-1,-1,-1],
                             [-1, 1,-1],
                             [ 1,-1,-1]]